In [1]:
import geopy.distance
import pandas as pd
import numpy as np
import os
import csv
#import shapefile
import matplotlib.pyplot as plt

## 1 - Creating the new dataframe

In [2]:
csv_file_path = 'Downloads/distance_matrix_by_tracts_OR.csv'
df = pd.read_csv(csv_file_path)

In [3]:
df.head()

,Unnamed: 0,origin,destination,distance
0,1,41029001900,6001400100,299.483284
1,2,41029002100,6001400100,299.948081
2,3,41029002200,6001400100,298.366958
3,4,41029002300,6001400100,294.933344
4,5,41035970600,6001400100,290.683531


In [4]:
len(df)

1846410

In [7]:
# First check - make sure there are no internal trips (i.e. tract in OR to tract in OR)

In [10]:
val1 = 41029001900
val2 = 41029001900
row_exists = (df['origin'] == val1) & (df['destination'] == val2)

In [11]:
if row_exists.any():
    print("Row exists")
else:
    print("Row does not exist")

Row exists


In [12]:
# Need to flag destinations in California
# State GEOID is 53. Hence numbers between 6000000000 and 6999999999 are in California
df['dest_OR'] = np.where((df['destination'] >= 41000000000) & (df['destination'] <= 41999999999) , 1, 0)

In [13]:
df.head()

,Unnamed: 0,origin,destination,distance,dest_OR
0,1,41029001900,6001400100,299.483284,0
1,2,41029002100,6001400100,299.948081,0
2,3,41029002200,6001400100,298.366958,0
3,4,41029002300,6001400100,294.933344,0
4,5,41035970600,6001400100,290.683531,0


In [14]:
# Filter out WA destinations
df_out=df[df['dest_OR']==0]

In [15]:
# Calculate the minimum distance for every origin
# Output: origin tract - minimum distance to out of state tract - tract
min_dist_out = df.loc[df_out.groupby('origin')['distance'].idxmin()]

#print(min_dist_out)

In [16]:
min_dist_out.head()

,Unnamed: 0,origin,destination,distance,dest_OR
164891,164892,41001950100,16087970300,54.874758,0
164892,164893,41001950200,16087970300,46.856612,0
164893,164894,41001950300,16087970300,26.776827,0
164894,164895,41001950400,16087970300,44.792487,0
164895,164896,41001950500,16087970300,49.020114,0


In [17]:
val1 = 41001950100
row_exists = (min_dist_out['origin'] == val1)
if row_exists.any():
    print("Row exists")
else:
    print("Row does not exist")

Row exists


In [18]:
min_dist_out = min_dist_out.reset_index()
final_df = min_dist_out[['origin','destination','distance']].copy()

In [19]:
final_df.head()

,origin,destination,distance
0,41001950100,16087970300,54.874758
1,41001950200,16087970300,46.856612
2,41001950300,16087970300,26.776827
3,41001950400,16087970300,44.792487
4,41001950500,16087970300,49.020114


In [20]:
len(final_df)

828

In [21]:
# Flag: California has 838 tracts

In [22]:
final_df.to_csv('tract_to_border_distance_OR.csv')

## 2 - Testing the output

Where are the destinations?

In [18]:
final_df['First2'] = final_df['destination'].astype(str).str[:2]

In [19]:
final_df['First2'].value_counts()

32    6462
40    1461
41     115
Name: First2, dtype: int64

In [20]:
# 32 is Nevada, # 41 is Oregon, #4 is Arizona

Most common destinations

In [22]:
dest_count_df = final_df['destination'].value_counts().reset_index()
dest_count_df.columns = ['GEOID', 'Count']

In [23]:
dest_count_df['GEOID'] = dest_count_df['GEOID'].astype(str)
dest_count_df.head()

,GEOID,Count
0,32023960405,2903
1,32005001800,1206
2,32005990000,1191
3,4012020602,789
4,4027011403,423
